In [82]:
#V:1.0 - Coded by Alexandre Ménard - 2024
import ipywidgets as W
import pandas as pd
from IPython.display import display
from numpy import exp, sqrt, log
from scipy import stats
from math import *

def european_opt_pricing(b):
    output.clear_output()
    if (call_checkbox.value == True and put_checkbox.value == False) or (call_checkbox.value == False and put_checkbox.value == True):

        if call_checkbox.value == True and put_checkbox.value == False:
            type_opt = 'Call'
        elif call_checkbox.value == False and put_checkbox.value == True:
            type_opt = 'Put'
        
        spot = float(stock_price_box.value)
        strike = float(strike_price_box.value)
        volatility = float(volatility_box.value)
        interest_rate = float(interest_rate_box.value)
        maturity = float(maturity_box.value)
        dividend = float(dividend_rate_box.value)

        d1 = ((log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*sqrt(maturity)))
        d2 = d1-volatility*sqrt(maturity)
        d1RC = (1/(sqrt(2*pi)))*exp(-(d1**2)/2)

        if type_opt == 'Call':
            price_opt = round(spot*stats.norm.cdf(d1)*exp(-dividend*maturity)-strike*(exp(-interest_rate*maturity))*stats.norm.cdf(d2),2)
        elif type_opt == 'Put':
            price_opt = round(strike*(exp(-interest_rate*maturity))*stats.norm.cdf(-d2)-spot*stats.norm.cdf(-d1)*exp(-dividend*maturity),2)
    
        #Greeks (1st order)
        if type_opt == 'Call':
            delta = round(stats.norm.cdf(d1)*exp(-dividend*maturity),4)
        elif type_opt == 'Put':
            delta = round(-stats.norm.cdf(-d1)*exp(-dividend*maturity),4)
            
        gamma = round((d1RC*exp(-dividend*maturity))/(spot*volatility*sqrt(maturity)),4)    
        vega = round((d1RC*exp(-dividend*maturity)*sqrt(maturity)*spot)/100,4)
        
        if type_opt == 'Call':
            rho = round((strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(d2))/100,4)
        elif type_opt == 'Put':
            rho = round((-strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(-d2))/100,4)

        if type_opt == 'Call':
            theta = round((-((d1RC*(spot*exp(-dividend*maturity))*volatility)/(2*sqrt(maturity)))-interest_rate*(strike*exp(-interest_rate*maturity))*stats.norm.cdf(d2)+dividend*(spot*exp(-dividend*maturity))*stats.norm.cdf(d1))/252,4)
        elif type_opt == 'Put':          
            theta = round((-((d1RC*(spot*exp(-dividend*maturity))*volatility)/(2*sqrt(maturity)))+interest_rate*(strike*exp(-interest_rate*maturity))*stats.norm.cdf(-d2)-dividend*(spot*exp(-dividend*maturity))*stats.norm.cdf(-d1))/252,4)
            
        #Results
        Price_box.value = str(price_opt)
        Delta_box.value = str(delta)
        Gamma_box.value = str(gamma)
        Vega_box.value = str(vega)
        Rho_box.value = str(rho)
        Theta_box.value = str(theta)
        
        with output:
            print("Param : " + str(spot) + "," + str(strike) + "," + str(volatility) + "," + str(interest_rate) + "," + str(maturity) + "," + str(dividend))
            print("Pricing...")
            print("Final price : " + str(price_opt))
            chart_button.disabled=False
            #print("D1 prime : " + str(d1RC)) #For checking..
            #print(type(spot)) #For checking..
    else:
        with output:
            print("Please select call or put option, not both!")
            chart_button.disabled=True

stock_price_box = W.Combobox(value='100', placeholder='100', description='Stock Price :')
strike_price_box = W.Combobox(value='100', placeholder='100', description='Strike Price :')
volatility_box = W.Combobox(value='0.2', placeholder='0.2', description='Volatility :')
interest_rate_box = W.Combobox(value='0.02', placeholder='0.03', description='Interest rate :')
maturity_box = W.Combobox(value='1', placeholder='1.5', description='Maturity :')
dividend_rate_box = W.Combobox(value='0.05', placeholder='0.05', description='Dividend yield :')
price_button = W.Button(description='Price !',disabled=False)
chart_button = W.Button(description='Get Chart',disabled=True)
output = W.Output()
call_checkbox = W.Checkbox(value=False,description='Call',disabled=False,indent=False)
put_checkbox = W.Checkbox(value=False,description='Put',disabled=False,indent=False)
Price_box = W.Text(value='',placeholder='Please price',description='Final Price:',disabled=True)
Delta_box = W.Text(value='',placeholder='Please price',description='Delta:',disabled=True)
Gamma_box = W.Text(value='',placeholder='Please price',description='Gamma:',disabled=True)
Theta_box = W.Text(value='',placeholder='Please price',description='Theta:',disabled=True)
Vega_box = W.Text(value='',placeholder='Please price',description='Vega:',disabled=True)
Rho_box = W.Text(value='',placeholder='Please price',description='Rho:',disabled=True)

type_opt = W.HBox([call_checkbox, put_checkbox])
line1 = W.HBox([stock_price_box, Price_box])
line2 = W.HBox([strike_price_box, Delta_box, chart_button])
line3 = W.HBox([volatility_box, Gamma_box, chart_button])
line4 = W.HBox([interest_rate_box, Theta_box, chart_button])
line5 = W.HBox([dividend_rate_box, Vega_box, chart_button])
line6 = W.HBox([maturity_box, Rho_box, chart_button])
#type_opt = W.HBox([W.Checkbox(value=False,description='Call',disabled=False,indent=False),W.Checkbox(value=False,description='Put',disabled=False,indent=False)])

display(type_opt)
display(line1)
display(line2)
display(line3)
display(line4)
display(line5)
display(line6)
display(price_button,output)


def on_button_clicked(b):
    with output:
        print("Button clicked.")

price_button.on_click(european_opt_pricing)    

    

Button(description='Price !', style=ButtonStyle())

Output()